In [1]:
import pandas as pd

In [2]:
import os
os.chdir('data')

In [3]:
from datetime import datetime, timedelta, date, time

# Загрузка данных

In [111]:
def get_period(start_date, finish_date, period='m'):
    period_values = []
    i = 0
    curr_date = start_date
    if period == 'm':
        period_values.append(curr_date)
        while curr_date < finish_date:
            y = (start_date.month + i) // 12
            m = (start_date.month + i) % 12 + 1
            curr_date = date(start_date.year + y, m, 1)
            period_values.append(curr_date)
            i = i + 1
    elif period == 'w':
        while curr_date <= finish_date:
            period_values.append(curr_date)
            curr_date = curr_date + timedelta(days=7)
    return period_values

In [5]:
def load_from_csv(filename, start_date, finish_date, period='m',
                  selected_locs=None, selected_cols=['LOCATION', 'TIME', 'Value'], sep=','):
    df = pd.read_csv(f"{filename}.csv", sep=sep, parse_dates=[selected_cols[1]])
    df = df[selected_cols]
    df.rename(columns={selected_cols[2]: filename, selected_cols[1]: "period"}, inplace=True)
    df.set_index('period', inplace=True)
    
    period_df = pd.DataFrame(data={'period': get_period(start_date, finish_date, period=period)})
    period_df.set_index('period', inplace=True)
    if not selected_locs:
        selected_locs = list(df[selected_cols[0]].unique())
    for col in selected_locs:
        loc_df = df[(df[selected_cols[0]] == col)][[filename]]
        loc_df.rename(columns={filename: f"{filename}_{col}"}, inplace=True)
        period_df = period_df.join(loc_df, how='left')
    return period_df

In [6]:
def fill_nan_last_val(df, sym=None):
    new_df = df.copy()
    for col in df.columns:
        if not sym:
            f_isna = (new_df[col].isna())
            f_notna = (df[col].notna())
        else:
            f_isna = (new_df[col] == '..')
            f_notna = (new_df[col] != '..')
        for index, value in new_df.loc[f_isna][col].items():
            s = df[((df[col].notna()) & (df.index <= index))][col]
            if len(s):
                last_val = s.iloc[-1]
            else:
                last_val = df[(f_notna & (df.index > index))][col].iloc[0]
            df.loc[index, col] = last_val
            df[col] = df[col].astype('float64', copy=False)
    #return df

In [121]:
def get_df_from_excel(filename, start_date, finish_date,
                      skiprows, skipfooter, cols_rename, period='m', parse_dates=['Date'],
                      select_cols=None):
    df = pd.read_excel(f"{filename}.xlsx", decimal=',', 
                       parse_dates=parse_dates, skiprows=skiprows, skipfooter=skipfooter)
    if select_cols:
        df = df[select_cols]
    df.rename(columns=cols_rename, inplace=True)
    df.set_index('period', inplace=True)
    
    period_df = pd.DataFrame(data={'period': get_period(start_date, finish_date, period=period)})
    period_df.set_index('period', inplace=True)
    return period_df.join(df, how='left')

# Параметры

In [8]:
start_date = date(2000, 1, 1)
finish_date = date(2022, 6, 1)

### Обработка BCI

In [9]:
bci_df = load_from_csv('BCI', start_date, finish_date)

In [10]:
fill_nan_last_val(bci_df)

### CCI

In [11]:
cci_df = load_from_csv('CCI', start_date, finish_date)
fill_nan_last_val(cci_df)

In [12]:
cci_df.tail()

,CCI_JPN,CCI_AUS,CCI_OECD,CCI_GBR,CCI_ZAF,CCI_USA,CCI_BRA,CCI_CHN,CCI_G-7,CCI_IDN,CCI_EA19,CCI_RUS
period,,,,,,,,,,,,
2022-02-01,98.00504,99.86245,98.28129,97.40088,98.28765,97.08185,98.24409,102.7179,97.93880,100.9402,99.11861,97.62921
2022-03-01,97.58710,99.59343,97.72383,95.79521,98.06467,96.92998,98.33363,101.4161,97.39916,100.6590,97.51540,97.62921
2022-04-01,97.28867,99.35078,97.28673,94.11323,98.06467,97.05984,98.54461,101.4161,97.06406,100.6590,96.11227,97.62921
2022-05-01,97.28867,99.35078,97.28673,94.11323,98.06467,97.05984,98.54461,101.4161,97.06406,100.6590,96.11227,97.62921
2022-06-01,97.28867,99.35078,97.28673,94.11323,98.06467,97.05984,98.54461,101.4161,97.06406,100.6590,96.11227,97.62921


### CLI

In [13]:
cli_df = load_from_csv('CLI', start_date, finish_date)
fill_nan_last_val(cli_df)

In [14]:
cli_df.tail()

,CLI_GBR,CLI_CHN,CLI_BRA,CLI_OECD,CLI_IDN,CLI_JPN,CLI_USA,CLI_IND,CLI_AUS,CLI_CAN,CLI_ZAF,CLI_G-7,CLI_RUS,CLI_EA19
period,,,,,,,,,,,,,,
2022-02-01,100.6501,99.40155,98.78979,100.3172,96.53909,100.5638,100.0371,100.09300,98.95084,100.1341,101.0914,100.2271,103.0155,100.4592
2022-03-01,100.3470,99.38866,98.47284,100.2351,96.43018,100.5751,100.0779,100.01170,98.75658,100.0726,101.0999,100.1705,103.0155,100.2647
2022-04-01,100.0326,99.38767,98.25556,100.1643,96.36440,100.5995,100.1464,99.93071,98.59047,100.0274,101.0918,100.1361,103.0155,100.0879
2022-05-01,100.0326,99.38767,98.25556,100.1643,96.36440,100.5995,100.1464,99.93071,98.59047,100.0274,101.0918,100.1361,103.0155,100.0879
2022-06-01,100.0326,99.38767,98.25556,100.1643,96.36440,100.5995,100.1464,99.93071,98.59047,100.0274,101.0918,100.1361,103.0155,100.0879


### PPI Manufacturing domestic market

In [15]:
ppi_dm_df = load_from_csv('PPI Manufacturing domestic market', 
                          start_date, finish_date, selected_locs=['OECD'])
fill_nan_last_val(ppi_dm_df)

In [16]:
ppi_dm_df.tail()

,PPI Manufacturing domestic market_OECD
period,
2022-02-01,132.1458
2022-03-01,136.4849
2022-04-01,139.3227
2022-05-01,139.3227
2022-06-01,139.3227


### PPI Manufacturing total market

In [17]:
ppi_tm_df = load_from_csv('PPI Manufacturing total market', 
                          start_date, finish_date, selected_locs=['USA'])
fill_nan_last_val(ppi_tm_df)

In [18]:
ppi_tm_df.tail()

,PPI Manufacturing total market_USA
period,
2022-02-01,127.855
2022-03-01,132.132
2022-04-01,134.284
2022-05-01,134.284
2022-06-01,134.284


### Leading Indicators OECD  Component series  Interest rate spread  Original series

In [19]:
li_irs_df = load_from_csv('Leading Indicators OECD  Component series  Interest rate spread  Original series',
                           start_date, finish_date,
                           selected_locs=['USA'],
                           selected_cols=['LOCATION', 'TIME', 'Value'],
                           sep='\t')
fill_nan_last_val(li_irs_df)

In [20]:
li_irs_df.tail()

,Leading Indicators OECD Component series Interest rate spread Original series_USA
period,
2022-02-01,1.85
2022-03-01,1.93
2022-04-01,2.42
2022-05-01,2.42
2022-06-01,2.42


### Leading Indicators OECD > Component series > Share prices > Normalised-Level

In [21]:
li_sp_df = load_from_csv('Leading Indicators OECD  Component series  Share prices  Normalised-Level',
                         start_date, finish_date,
                         selected_locs=['USA'],
                         selected_cols=['LOCATION', 'TIME', 'Value'],
                         sep='\t')
fill_nan_last_val(li_sp_df)

In [22]:
li_sp_df.tail()

,Leading Indicators OECD Component series Share prices Normalised-Level_USA
period,
2022-02-01,100.344766
2022-03-01,100.167408
2022-04-01,100.003445
2022-05-01,100.003445
2022-06-01,100.003445


### Industrial Production

In [23]:
ip_files = ['Industrial Production  Brazil', 'Industrial Production  Russia', 
            'Industrial Production E. Europe', 'Industrial Production South East Asia Average']

In [24]:
ip_dfs = list(map(lambda filename: 
                  get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=7, 
                  cols_rename={"% Yr/Yr": filename, "Date": "period"}), ip_files))
ip_df = pd.concat(ip_dfs, axis=1, join='inner')
fill_nan_last_val(ip_df)

In [25]:
ip_df.tail()

,Industrial Production Brazil,Industrial Production Russia,Industrial Production E. Europe,Industrial Production South East Asia Average
period,,,,
2022-02-01,-4.3,6.3,6.53662,5.77101
2022-03-01,-2.1,3.1,6.53662,5.77101
2022-04-01,-2.1,3.1,6.53662,5.77101
2022-05-01,-2.1,3.1,6.53662,5.77101
2022-06-01,-2.1,3.1,6.53662,5.77101


### US

In [26]:
us_files = ['US Total Coking Coal Exports', 
            'US Seaborne Coking Coal Exports']

In [27]:
df_us_coking_coal_exp = \
get_df_from_excel(us_files[0], start_date, finish_date,
                  skiprows=5, skipfooter=6, 
                  cols_rename={",000 tonnes": us_files[0], "Date": "period"})

In [28]:
df_us_seaborne_coal_exp = \
get_df_from_excel(us_files[1], start_date, finish_date,
                  skiprows=5, skipfooter=7, 
                  cols_rename={",000 tonnes": us_files[1], "Date": "period"})

In [29]:
us_df = pd.concat([df_us_coking_coal_exp, df_us_seaborne_coal_exp], axis=1, join='inner')
fill_nan_last_val(us_df)

In [30]:
us_df.tail()

,US Total Coking Coal Exports,US Seaborne Coking Coal Exports
period,,
2022-02-01,3627.0,3626.0
2022-03-01,3627.0,3626.0
2022-04-01,3627.0,3626.0
2022-05-01,3627.0,3626.0
2022-06-01,3627.0,3626.0


### Exports Imports in goods OECD

In [31]:
def replace_non_val(df, cols, sym='..'):
    for col in cols:
        df.loc[(df[col] == '..'), col] = 0
        df[col] = df[col].astype('float64', copy=False)
    return df

In [32]:
filename = 'Exports Imports in goods OECD'

selected_cols = ['Subject', 'LOCATION', 'TIME', 'Value']
exp_imp_in_goods_df = pd.read_csv(f"{filename}.csv", sep='\t', parse_dates=['TIME'])
exp_imp_in_goods_df = exp_imp_in_goods_df[selected_cols]
exp_imp_in_goods_df.rename(columns={'TIME': "period"}, inplace=True)
exp_imp_in_goods_df = exp_imp_in_goods_df.set_index('period')

In [33]:
period_df = pd.DataFrame(data={'period': get_period(start_date, finish_date)})
period_df.set_index('period', inplace=True)
exp_in_goods_df = exp_imp_in_goods_df[(exp_imp_in_goods_df.Subject == 'Exports in goods (value)')]
exp_in_goods_df = exp_in_goods_df[['LOCATION', 'Value']]
locs = exp_in_goods_df['LOCATION'].unique()

exp_in_goods_locs = []
for loc in locs:
    loc_df = exp_in_goods_df[(exp_in_goods_df['LOCATION'] == loc)][['Value']]
    loc_df.rename(columns={'Value': f"Exports in goods (value)-US-Dollar_{loc}"}, inplace=True)
    loc_df = period_df.join(loc_df, how='left')
    exp_in_goods_locs.append(loc_df)
exp_in_goods_df = pd.concat(exp_in_goods_locs, axis=1, join='inner')

fill_nan_last_val(exp_in_goods_df, sym='..')
fill_nan_last_val(exp_in_goods_df)
exp_in_goods_df['Exports in goods (value)-US-Dollar_OECD_Average'] = \
exp_in_goods_df[exp_in_goods_df.columns[:-1]].mean(axis=1)

In [34]:
exp_in_goods_df.tail()

,Exports in goods (value)-US-Dollar_AUS,Exports in goods (value)-US-Dollar_CAN,Exports in goods (value)-US-Dollar_FRA,Exports in goods (value)-US-Dollar_DEU,Exports in goods (value)-US-Dollar_ITA,Exports in goods (value)-US-Dollar_JPN,Exports in goods (value)-US-Dollar_KOR,Exports in goods (value)-US-Dollar_MEX,Exports in goods (value)-US-Dollar_CHE,Exports in goods (value)-US-Dollar_TUR,...,Exports in goods (value)-US-Dollar_POL,Exports in goods (value)-US-Dollar_PRT,Exports in goods (value)-US-Dollar_SVK,Exports in goods (value)-US-Dollar_SWE,Exports in goods (value)-US-Dollar_EST,Exports in goods (value)-US-Dollar_LVA,Exports in goods (value)-US-Dollar_LTU,Exports in goods (value)-US-Dollar_SVN,Exports in goods (value)-US-Dollar_OECD,Exports in goods (value)-US-Dollar_OECD_Average
period,,,,,,,,,,,,,,,,,,,,,
2022-02-01,29.41692,42.13962,50.62971,139.6229,53.60439,62.43734,53.99846,46.23926,25.83490,19.91107,...,28.97130,6.769989,9.150682,16.29527,1.691987,1.752678,3.345971,5.010164,1070.440,28.169484
2022-03-01,32.82041,54.05064,56.11316,151.8868,62.10876,71.36413,63.79319,51.90614,26.99028,22.68892,...,30.24796,7.278731,9.503868,19.10878,2.181051,2.067640,4.062317,6.175365,1231.299,32.402598
2022-04-01,33.33560,54.05064,56.11316,151.8868,62.10876,63.92207,57.83341,47.47947,22.73160,23.36755,...,30.24796,7.278731,9.503868,16.32535,2.181051,2.067640,4.062317,6.175365,1231.299,31.224002
2022-05-01,33.33560,54.05064,56.11316,151.8868,62.10876,63.92207,57.83341,47.47947,22.73160,23.36755,...,30.24796,7.278731,9.503868,16.32535,2.181051,2.067640,4.062317,6.175365,1231.299,31.224002
2022-06-01,33.33560,54.05064,56.11316,151.8868,62.10876,63.92207,57.83341,47.47947,22.73160,23.36755,...,30.24796,7.278731,9.503868,16.32535,2.181051,2.067640,4.062317,6.175365,1231.299,31.224002


In [35]:
period_df = pd.DataFrame(data={'period': get_period(start_date, finish_date)})
period_df.set_index('period', inplace=True)
imp_in_goods_df = exp_imp_in_goods_df[(exp_imp_in_goods_df.Subject == 'Imports in goods (value)')]
imp_in_goods_df = imp_in_goods_df[['LOCATION', 'Value']]
locs = imp_in_goods_df['LOCATION'].unique()

imp_in_goods_locs = []
for loc in locs:
    loc_df = imp_in_goods_df[(imp_in_goods_df['LOCATION'] == loc)][['Value']]
    loc_df.rename(columns={'Value': f"Imports in goods (value)-US-Dollar_{loc}"}, inplace=True)
    loc_df = period_df.join(loc_df, how='left')
    imp_in_goods_locs.append(loc_df)
imp_in_goods_df = pd.concat(imp_in_goods_locs, axis=1, join='inner')

fill_nan_last_val(imp_in_goods_df, sym='..')
fill_nan_last_val(imp_in_goods_df)

imp_in_goods_df['Imports in goods (value)-US-Dollar_OECD_Average'] = \
imp_in_goods_df[imp_in_goods_df.columns[:-1]].mean(axis=1)

In [36]:
imp_in_goods_df.tail()

,Imports in goods (value)-US-Dollar_AUS,Imports in goods (value)-US-Dollar_CAN,Imports in goods (value)-US-Dollar_FRA,Imports in goods (value)-US-Dollar_DEU,Imports in goods (value)-US-Dollar_ITA,Imports in goods (value)-US-Dollar_JPN,Imports in goods (value)-US-Dollar_KOR,Imports in goods (value)-US-Dollar_MEX,Imports in goods (value)-US-Dollar_TUR,Imports in goods (value)-US-Dollar_GBR,...,Imports in goods (value)-US-Dollar_POL,Imports in goods (value)-US-Dollar_PRT,Imports in goods (value)-US-Dollar_SVK,Imports in goods (value)-US-Dollar_SWE,Imports in goods (value)-US-Dollar_EST,Imports in goods (value)-US-Dollar_LVA,Imports in goods (value)-US-Dollar_LTU,Imports in goods (value)-US-Dollar_SVN,Imports in goods (value)-US-Dollar_OECD,Imports in goods (value)-US-Dollar_OECD_Average
period,,,,,,,,,,,,,,,,,,,,,
2022-02-01,22.65962,39.89195,64.59635,127.3428,55.60778,68.31826,53.09772,44.95308,27.88449,62.82201,...,30.87780,9.300313,9.324702,16.15535,2.030271,2.045586,3.961167,5.486394,1179.396,31.036736
2022-03-01,25.92466,50.98204,73.65316,142.2904,62.20090,74.94086,63.58105,51.80168,30.87621,76.06483,...,34.38326,9.940431,9.817210,18.61774,2.392003,2.556114,4.897969,6.386648,1383.783,36.415350
2022-04-01,22.70632,50.98204,73.65316,142.2904,62.20090,70.59300,60.34150,49.36353,29.48048,65.10110,...,34.38326,9.940431,9.817210,16.51420,2.392003,2.556114,4.897969,6.386648,1383.783,35.260327
2022-05-01,22.70632,50.98204,73.65316,142.2904,62.20090,70.59300,60.34150,49.36353,29.48048,65.10110,...,34.38326,9.940431,9.817210,16.51420,2.392003,2.556114,4.897969,6.386648,1383.783,35.260327
2022-06-01,22.70632,50.98204,73.65316,142.2904,62.20090,70.59300,60.34150,49.36353,29.48048,65.10110,...,34.38326,9.940431,9.817210,16.51420,2.392003,2.556114,4.897969,6.386648,1383.783,35.260327


## Fleet & Supply

In [37]:
filename = 'Total Bulkcarrier Demolition'
dem_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=8, 
                  cols_rename={"DWT million": 'Total Bulkcarrier Demolition_30234',
                               'No': 'Total Bulkcarrier Demolition_30233',
                               "Date": "period"})
fill_nan_last_val(dem_df)

## Commodity prices

In [38]:
filename = 'Commodity Prices_m'
commodity_prices_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=11, cols_rename={"Date": "period"})
fill_nan_last_val(commodity_prices_df)

In [39]:
commodity_prices_df.tail()

,Brent Crude Oil Price,Japan Steel Ship Plate Commodity Price,Saudi Posted Contract Price - Propane,Saudi Posted Contract Price - Butane,Mont Belvieu Texas Spot Price - Propane,"Gasoline Spot Price, FOB US Gulf","Natural Gas Spot Price, Henry Hub USA","Natural Gas Spot Price, NGB Point Europe","Coking Coal Price, FOB Australia","Natural Gas Spot Price, TTF NW Europe"
period,,,,,,,,,,
2022-02-01,94.10150,1000.0,775.0,775.0,127.975,2.64475,4.7425,29.47959,398.8125,29.63262
2022-03-01,112.46304,1000.0,895.0,920.0,145.150,3.17450,4.7725,39.48221,574.1875,41.06364
2022-04-01,105.91700,1000.0,940.0,960.0,132.060,3.17360,6.4060,29.05594,415.5000,31.60211
2022-05-01,111.95955,1000.0,850.0,860.0,122.300,3.74000,8.1275,21.14925,469.0000,27.40366
2022-06-01,111.95955,1000.0,850.0,860.0,122.300,3.74000,8.1275,21.14925,469.0000,27.40366


## Inflation and LIBOR

In [40]:
filename = 'LIBOR Interest Rates_m_14948'
libor_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=6, cols_rename={'%': 'LIBOR Interest Rates_m_14948', "Date": "period"})
fill_nan_last_val(libor_df)

In [41]:
filename = 'Inflation Indicator'
inflation_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=6, cols_rename={"Date": "period"})
fill_nan_last_val(inflation_df)

In [42]:
inflation_df.tail()

,Inflation Indicator OECD Europe (excl Turkey),Inflation Indicator USA,Inflation Indicator Japan,Inflation Indicator OECD
period,,,,
2022-02-01,9.8,7.9,0.9,7.8
2022-03-01,11.6,8.5,1.2,8.8
2022-04-01,12.6,8.3,2.5,9.2
2022-05-01,12.6,8.3,2.5,9.2
2022-06-01,12.6,8.3,2.5,9.2


## Export & Import

In [43]:
filename = 'Indonesia Steam Coal Exports (including lignite)'
ind_steam_coal_ex_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=7, 
                  cols_rename={",000 tonnes": filename, "Date": "period"})
fill_nan_last_val(ind_steam_coal_ex_df)

In [44]:
filename = 'Handysize 28k dwt 20 Year Old Secondhand Prices_47316'
cols_rename = {"Date": "period", '$m':filename}
vessel_yo_20_price_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=7, 
                  cols_rename=cols_rename, parse_dates=['Date'])
fill_nan_last_val(vessel_yo_20_price_df)

In [45]:
filename = 'China Seaborne Coking Coal Imports'
cols_rename = {"Date": "period", ',000 tonnes': filename}
china_coal_imp_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=7, 
                  cols_rename=cols_rename, parse_dates=['Date'])
fill_nan_last_val(china_coal_imp_df)

In [46]:
filename = 'China Coking Coal Exports'
cols_rename = {"Date": "period", ',000 tonnes': filename}
china_coal_exp_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=7, 
                  cols_rename=cols_rename, parse_dates=['Date'])
fill_nan_last_val(china_coal_exp_df)

In [47]:
filename = 'Australia Steam Coal Exports (As Reported)'
cols_rename = {"Date": "period", ',000 tonnes': filename}
australia_coal_exp_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=7, 
                  cols_rename=cols_rename, parse_dates=['Date'])
fill_nan_last_val(australia_coal_exp_df)

In [48]:
filename = '25-30K DWT Handysize Bulkcarrier Newbuilding Prices'
cols_rename = {"Date": "period", '$m': filename}
vessel_new_price_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=8, 
                  cols_rename=cols_rename, parse_dates=['Date'])
fill_nan_last_val(vessel_new_price_df)

In [76]:
filename = 'Target_SIN_Timeseries_20220530192831'
cols_rename = {"Date": "period", '$/day': 'target'}
one_year_timecharter_rate_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=7, 
                  cols_rename=cols_rename, parse_dates=['Date'])
#fill_nan_last_val(one_year_timecharter_rate_df)

In [77]:
filename = '1 Year Timecharter Rate Handysize Bulkcarrier (Long Run Historical Series)_w_534426'
cols_rename = {"Date": "period", '$/day': 'target'}
one_year_timecharter_rate_w_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=7, period='w',
                  cols_rename=cols_rename, parse_dates=['Date'])

In [78]:
last_year = one_year_timecharter_rate_w_df.index[-1].year
last_month = one_year_timecharter_rate_w_df.index[-1].month
f = ((one_year_timecharter_rate_w_df.index.year == last_year) &
     (one_year_timecharter_rate_w_df.index.month == last_month))
last_value = one_year_timecharter_rate_w_df[f].mean(axis=0).target

In [79]:
f = ((one_year_timecharter_rate_df.index.year == last_year) &
     (one_year_timecharter_rate_df.index.month == last_month) &
     (one_year_timecharter_rate_df.target.isna()))
if one_year_timecharter_rate_df[f].shape[0]:
    one_year_timecharter_rate_df.loc[f, 'target'] = last_value

In [53]:
filename = 'Handysize Bulker Sales'
cols_rename = {"Date": "period", '$m': 'Handysize Bulker Sales_33262'}
handysize_bulker_sales_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=10, 
                  cols_rename=cols_rename, parse_dates=['Date'],
                  select_cols=['Date', '$m'])
fill_nan_last_val(handysize_bulker_sales_df)

In [54]:
filename = 'Handymax Bulker Scrap Value'
cols_rename = {"Date": "period", '$m': 'Handymax Bulker Scrap Value_532217'}
handymax_bulker_scrap_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=10, 
                  cols_rename=cols_rename, parse_dates=['Date'],
                  select_cols=['Date', '$m'])
fill_nan_last_val(handymax_bulker_scrap_df)

In [55]:
filename = 'Handysize Bulker Scrap Value'
cols_rename = {"Date": "period", '$m': 'Handysize Bulker Scrap Value_8693'}
handysize_bulker_scrap_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=7, 
                  cols_rename=cols_rename, parse_dates=['Date'],
                  select_cols=['Date', '$m'])
fill_nan_last_val(handysize_bulker_scrap_df)

## Weekly data

In [115]:
start_date = date(2000, 1, 7)
finish_date = date(2022, 6, 24)

In [129]:
filename = 'Commodity Prices_w'
commodity_prices_w_df = \
get_df_from_excel(filename, start_date, finish_date, period='w', 
                  skiprows=5, skipfooter=7, cols_rename={"Date": "period"})
fill_nan_last_val(commodity_prices_w_df)

In [130]:
commodity_prices_w_df.tail()

,Brent Crude Oil Price,"Natural Gas Spot Price, TTF NW Europe"
period,,
2022-05-27,115.5680,25.31536
2022-06-03,119.6260,25.42693
2022-06-10,121.7480,24.65798
2022-06-17,118.9760,31.45380
2022-06-24,112.6425,31.45380


In [131]:
filename = 'Secondhand Prices_w'
secondhand_prices_w_df = \
get_df_from_excel(filename, start_date, finish_date, period='w', 
                  skiprows=5, skipfooter=11, cols_rename={"Date": "period"})
#fill_nan_last_val(secondhand_prices_w_df)

In [132]:
secondhand_prices_w_df.tail()

,Handysize 28K DWT 15 Year Old Secondhand Prices,Handysize 28k dwt 20 Year Old Secondhand Prices,Handysize 37k dwt 5 Year Old Secondhand Prices,Handysize 32K 10 Year Old Secondhand Prices,Handysize 38k dwt Resale Secondhand Prices
period,,,,,
2022-05-27,12.75,10.25,28.5,19.5,32.5
2022-06-03,12.75,10.25,28.5,19.5,32.5
2022-06-10,13.75,10.75,28.5,19.5,32.5
2022-06-17,13.75,10.75,28.5,19.5,32.5
2022-06-24,13.75,10.75,28.5,19.5,32.5


In [133]:
filename = 'Scrap Prices_w'
scrap_prices_w_df = \
get_df_from_excel(filename, start_date, finish_date, period='w', 
                  skiprows=5, skipfooter=8, cols_rename={"Date": "period"})
#fill_nan_last_val(secondhand_prices_w_df)

In [134]:
scrap_prices_w_df.tail()

,India Scrap Price (Handymax/Handysize Bulkcarrier),Bangladesh Scrap Price (Handymax/Handysize Bulkcarrier)
period,,
2022-05-27,590.0,635.0
2022-06-03,590.0,615.0
2022-06-10,590.0,615.0
2022-06-17,590.0,615.0
2022-06-24,585.0,600.0


In [137]:
filename = 'Interest Rates_w'
interest_rates_w_df = \
get_df_from_excel(filename, start_date, finish_date, period='w', 
                  skiprows=5, skipfooter=7, cols_rename={"Date": "period"})
fill_nan_last_val(interest_rates_w_df)

In [138]:
interest_rates_w_df.tail()

,US Interest Rate: Prime,Japan Interest Rate: Short Term Prime,UK Interest Rate: Official Bank Rate,Switzerland Interest Rate: Policy Rate,Market Interest Rate: US$ LIBOR 6 Months
period,,,,,
2022-05-27,4.0,1.475,2.0,0.00,2.054
2022-06-03,4.0,1.475,2.0,0.00,2.069
2022-06-10,4.0,1.475,2.0,0.00,2.266
2022-06-17,4.0,1.475,2.0,0.25,2.748
2022-06-24,4.0,1.475,2.0,0.25,2.748


In [139]:
filename = 'Handysize Bulkcarrier Newbuilding Prices_w'
newbuild_prices_w_df = \
get_df_from_excel(filename, start_date, finish_date, period='w', 
                  skiprows=5, skipfooter=8, cols_rename={"Date": "period"})
fill_nan_last_val(newbuild_prices_w_df)

In [140]:
newbuild_prices_w_df.tail()

,38-40K DWT Handysize Bulkcarrier Newbuilding Prices,25-30K DWT Handysize Bulkcarrier Newbuilding Prices
period,,
2022-05-27,30.0,26.93182
2022-06-03,30.0,26.93182
2022-06-10,30.0,26.93182
2022-06-17,30.0,26.93182
2022-06-24,30.5,27.38068


In [141]:
filename = '1 Year Timecharter Rate Handysize Bulkcarrier (Long Run Historical Series)_w'
cols_rename = {"Date": "period", '$/day': 'target'}
one_year_timecharter_rate_w_df = \
get_df_from_excel(filename, start_date, finish_date,
                  skiprows=5, skipfooter=7, period='w',
                  cols_rename=cols_rename, parse_dates=['Date'])

In [142]:
one_year_timecharter_rate_w_df.tail()

,target
period,
2022-05-27,26500.0
2022-06-03,26500.0
2022-06-10,26500.0
2022-06-17,24500.0
2022-06-24,23000.0


# Create dataset (month)

In [56]:
def get_lagged_df(df, col, n_lag=12):
    lag_period = 1
    df_new = pd.DataFrame()
    col_features = []
    for period_mult in range(1, n_lag + 1, 1):
        f_name = f"{col}_lag_{period_mult}"
        df_new[f_name] = df[col].shift(period_mult * lag_period)
        col_features.append(f_name)
    return df_new[col_features]

In [167]:
df_list =  [
            #bci_df,
            #ip_df,
            #us_df,
            dem_df,
            #ind_steam_coal_ex_df,
            vessel_yo_20_price_df,
            #china_coal_imp_df,
            #china_coal_exp_df,
            #australia_coal_exp_df,
            vessel_new_price_df, 
            #cci_df, 
            #cli_df,
            #ppi_tm_df,
            #ppi_dm_df,
            handysize_bulker_sales_df,
            #handymax_bulker_scrap_df,
            handysize_bulker_scrap_df,
            #li_irs_df,
            #li_sp_df,
            #exp_in_goods_df[['Exports in goods (value)-US-Dollar_OECD_Average']],
            #imp_in_goods_df[['Imports in goods (value)-US-Dollar_OECD_Average']],
            libor_df,
            inflation_df,
            commodity_prices_df
            ]

In [165]:
df_list =  [
            bci_df,
            ip_df,
            us_df,
            dem_df,
            ind_steam_coal_ex_df,
            vessel_yo_20_price_df,
            china_coal_imp_df,
            china_coal_exp_df,
            australia_coal_exp_df,
            vessel_new_price_df, 
            cci_df, 
            cli_df,
            ppi_tm_df,
            ppi_dm_df,
            handysize_bulker_sales_df,
            handymax_bulker_scrap_df,
            handysize_bulker_scrap_df,
            li_irs_df,
            li_sp_df,
            exp_in_goods_df[['Exports in goods (value)-US-Dollar_OECD_Average']],
            imp_in_goods_df[['Imports in goods (value)-US-Dollar_OECD_Average']],
            libor_df,
            inflation_df,
            commodity_prices_df
            ]

In [166]:
final_df = pd.concat([one_year_timecharter_rate_df] + df_list, axis=1)
final_df.to_csv('final_m_full_df.tsv', index=True, sep='\t')

In [168]:
final_df = pd.concat([one_year_timecharter_rate_df] + df_list, axis=1)
final_df.to_csv('final_m_df.tsv', index=True, sep='\t')

In [169]:
final_df.shape

(270, 22)

# Create dataset (week)

In [162]:
df_w_list = [
            commodity_prices_w_df,
            secondhand_prices_w_df,
            scrap_prices_w_df,
            interest_rates_w_df,
            newbuild_prices_w_df,
            ]

In [163]:
final_w_df = pd.concat([one_year_timecharter_rate_w_df] + df_w_list, axis=1)
final_w_df.to_csv('final_w_df.tsv', index=True, sep='\t')

In [164]:
final_w_df[(final_w_df.index >= '2009-09-04')].isna().any()

target                                                     False
Brent Crude Oil Price                                      False
Natural Gas Spot Price, TTF NW Europe                      False
Handysize 28K DWT 15 Year Old Secondhand Prices            False
Handysize 28k dwt 20 Year Old Secondhand Prices            False
Handysize 37k dwt 5 Year Old Secondhand Prices             False
Handysize 32K 10 Year Old Secondhand Prices                False
Handysize 38k dwt Resale Secondhand Prices                 False
India Scrap Price (Handymax/Handysize Bulkcarrier)         False
Bangladesh Scrap Price (Handymax/Handysize Bulkcarrier)    False
US Interest Rate: Prime                                    False
Japan Interest Rate: Short Term Prime                      False
UK Interest Rate: Official Bank Rate                       False
Switzerland Interest Rate: Policy Rate                     False
Market Interest Rate: US$ LIBOR 6 Months                   False
38-40K DWT Handysize Bulk